# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup and Imports

In [1]:
# For example: 
import os
from dotenv import load_dotenv                  # For loading environment variables from .env file
from datetime import datetime                   # For websearch metadata assignment

from typing import List, Dict, Any, Annotated
from pydantic import BaseModel, Field

from lib.agents import Agent                    # For Agent class
from lib.llm import LLM                         # For Large Language Model
from lib.messages import UserMessage, SystemMessage, BaseMessage
from lib.tooling import tool                    # For defining tools    
from lib.parsers import PydanticOutputParser    # To return structured output from LLM

from chromadb import PersistentClient           # For Vector Database
# import chromadb

from tavily import TavilyClient                 # For Web Search


In [2]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
VOCAREUM_OPENAI_API_KEY=os.getenv("VOCAREUM_OPENAI_API_KEY")
VOCAREUM_BASE_URL=os.getenv("VOCAREUM_BASE_URL")

Setting up ChromaDB paths and names. Should be consistent with what you used in the `Udaplay_01.ipynb` file.

In [3]:
CHROMADB_PATH = "chromadb"
CHROMADB_COLLECTION_NAME = "udaplay"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
@tool
def retrieve_game(query: str, n_results: int=4) -> List:
    """
#    Semantic search: Finds most relevant results in the vector DB
#    args:
#    - query: a question about game industry. 
#    - n_results: number of results to return (default is 4)
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
    """

    # collection.query(
        # query_texts=[query],
        # n_results=n_results,
        # include=['metadatas', 'documents', 'distances'] # TODO: look at collection.peek() to understand this better.
    # )

    # These have been created in Phase 1
    chroma_client = PersistentClient(path=CHROMADB_PATH)
    collection = chroma_client.get_collection(CHROMADB_COLLECTION_NAME)

    results = collection.query(
        query_texts=[query], 
        n_results=5
        )
    
    return results["documents"][0]

In [5]:
# To test the function
query = "Tell me about Gran Turismo"
response = retrieve_game(query)
print(response)

['[PlayStation 3] Gran Turismo 5 (2010, Sony Computer Entertainment) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.', '[PlayStation 1] Gran Turismo (1997, Sony Computer Entertainment) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.', "[PlayStation 2] Grand Theft Auto: San Andreas (2004, Rockstar Games) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.", '[Nintendo Switch] Mario Kart 8 Deluxe (2017, Nintendo) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.', "[PlayStation 4] Marvel's Spider-Man (2018, Sony Interactive Entertainment) - An open-world superhero game that lets players swing through New York City as Spider-Man, battling iconic villains."]


#### Evaluate Retrieval Tool

In [6]:
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Indicates if the retrieved documents are useful for answering the question")
    description: str = Field(..., description="Detailed explanation of the evaluation result, including why the documents are or are not useful")

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, this function 
    will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    Outputs:
    EvaluationReport:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    system_prompt = """
        You are an expert in evaluating the usability of documents for answering user queries.
        Your task is to evaluate if the provided sources are enough to respond the user query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.
    """

    user_prompt = f"""
        User query:\n{question}\n
        Retrieved documents:\n{retrieved_docs}
    """

    messages = [
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    llm_judge = LLM(
        model="gpt-4.1-nano",
        temperature=0.0,
        api_key=VOCAREUM_OPENAI_API_KEY,
        base_url=VOCAREUM_BASE_URL
    )

    ai_message = llm_judge.invoke(input=messages, response_format=EvaluationReport)
    parser = PydanticOutputParser(model_class=EvaluationReport)
    return parser.parse(ai_message)


In [7]:
# To test the function
response_valid = evaluate_retrieval(query, response)
print(response_valid.model_dump_json(indent=2))

{
  "useful": true,
  "description": "The retrieved documents include detailed descriptions of the Gran Turismo series, specifically mentioning its versions for PlayStation 1 and PlayStation 3, and highlighting its features as a realistic racing simulator with a wide selection of vehicles and tracks. This information directly addresses the user's query about Gran Turismo, providing relevant and sufficient details to answer their question."
}


#### Game Web Search Tool

In [8]:
class MetaData(BaseModel):
    timestamp: str = Field(..., description="The time the web search was performed, in ISO format (YYYY-MM-DD HH:MM:SS.mmmmmm)")
    query: str = Field(..., description="The original search query")

class WebSearchResult(BaseModel):
    answer: str = Field(..., description="Answer to the search query")
    results: List[Dict] = Field(..., description="List of search results")
    search_metadata: MetaData  = Field(..., description="Metadata of the web search")

# Modified from Course module 06 solution notebook (Web Search with Tavily)
@tool
def game_web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API.
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    Output: 
    WebSearchResult:
    - answer (str): Direct answer to the query, if available
    - results (List[Dict]): List of search results, each containing:
    - search_metadata (Dict): Metadata about the search, including timestamp and query
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    # metadata_parser = PydanticOutputParser(model_class=WebSearchResult)
    # validated_results = metadata_parser.parse(ai_message=AIMessage(content=str(formatted_results)))
    validated_results = WebSearchResult(**formatted_results)

    return validated_results

In [9]:
# To test the function
web_search_response = game_web_search(query)
print(web_search_response.model_dump_json(indent=2))

{
  "answer": "Gran Turismo is a 2023 film based on the true story of Jann Mardenborough, a gamer who became a professional race car driver. It follows his journey from winning a video game competition to racing in real life. The film stars Orlando Bloom and Archie Madekwe.",
  "results": [
    {
      "url": "https://en.wikipedia.org/wiki/Gran_Turismo_(film)",
      "title": "Gran Turismo (film) - Wikipedia",
      "content": "_Gran Turismo_( is a 2023 American biographicalsportsdrama film directed by Neill Blomkamp from a screenplay by Jason Hall \"Jason Hall (screenwriter)\") and Zach Baylin. Produced by Columbia Pictures, PlayStation Productions, and 2.0 Entertainment, it is based on the sim racing video game series \"Gran Turismo (series)\") developed by Polyphony Digital. It depicts a sensationalized account of real life British teenager Jann Mardenborough, a teenage _Gran Turismo_ player who became a professional [...] like a _Fast and Furious_ movie made without cynicism, and i

### Creating and invoking the Agent

In [15]:
tools = [retrieve_game, evaluate_retrieval, game_web_search]

instructions = f"""
    You are a helpful assistant that answers questions about video games. 
    Users will ask you about video games, including names, contents, publishing years, etc.

    You should first check whether an answer to the question is possible based on your knowledge (stored as a VectorDB), using the `retrieve_game` tool.
    After using `retrieve_game`, always run `evaluate_retrieval` to check whether the retrieved information is actually useful. 
    If the result of `evaluate_retrieval` is that the information is not useful, use your `game_web_search` tool to find an answer on the web. 

    Final output: 
    - In your final answer, always state whether the answer was from the curated database or from the open web.
    - If you found an answer on the web, cite the highest-ranked URL you found, so that users can go look up the primary source. The users don't have access to the search results, so cite a URL with link.
    - Do not make up facts.

    EXAMPLEs for final answer: 
    - Mario Kart is a racing game developed for the Nintendo GameCube. This information was retrieved from the internal database. 
    - Zelda: Tears of the Kingdom was released in 2023. This information was found on the web, see https://www.nintendo.com/en-gb/Games/Nintendo-Switch-games/The-Legend-of-Zelda-Tears-of-the-Kingdom-1576884.html. 
"""

web_agent = Agent(
    model_name="gpt-4.1-nano",
    instructions=instructions,
    tools=tools,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
)

In [11]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [12]:
query1="When were Pokémon Gold and Silver released?"

run1 = web_agent.invoke(
    query=query1
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)



[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = 
    You are a helpful assistant that answers questions about video games. 
    Users will ask you about video games, including names, contents, publishing years, etc.

    You should first check whether an answer to the question is possible based on your knowledge (stored as a VectorDB), using the `retrieve_game` tool.
    After using `retrieve_game`, always run `evaluate_retrieval` to check whether the retrieved information is actually useful. 
    If the result of `evaluate_retrieval` is that the information is not useful, use your `game_web_search` tool to find an answer on the web. 

    

In [13]:
query2="Which one was the first 3D platformer Mario game?"
run2 = web_agent.invoke(
    query=query2
)

print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = 
    You are a helpful assistant that answers questions about video games. 
    Users will ask you about video games, including names, contents, publishing years, etc.

    You should first check whether an answer to the question is possible based on your knowledge (stored as a VectorDB), using the `retrieve_game` tool.
    After using `retrieve_game`, always run `evaluate_retrieval` to check whether the retrieved information is actually useful. 
    If the result of `evaluate_retrieval` is that the information is not useful, use your `game_web_search` tool to find an answer on the web. 

    

In [16]:
query3="Was Mortal Kombat X realeased for Playstation 5?"

run3 = web_agent.invoke(
    query=query3
)

print("\nMessages from run 3:")
messages = run3.get_final_state()["messages"]
print_messages(messages)


[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = 
    You are a helpful assistant that answers questions about video games. 
    Users will ask you about video games, including names, contents, publishing years, etc.

    You should first check whether an answer to the question is possible based on your knowledge (stored as a VectorDB), using the `retrieve_game` tool.
    After using `retrieve_game`, always run `evaluate_retrieval` to check whether the retrieved information is actually useful. 
    If the result of `evaluate_retrieval` is that the infor

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes